<img src="images/deep_ga.png" align=right width=50%></img>
# Deep Neuroevolution
Author: Jin Yeom (jinyeom@utexas.edu)

## Contents
- [Configuration](#Configuration)
- [Genotype](#Genotype)
- [Phenotype](#Phenotype)

In [14]:
import random
from copy import deepcopy

import torch
from torch import nn
from torch.nn import functional as F
from torchsummary import summary

## Configuration

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device =", device)

device = cpu


In [9]:
SIGMA = 0.005

## Genotype

In [2]:
def rand_seed():
    return random.randint(0, 2**31-1)

In [4]:
class Genotype(object):
    def __init__(self):
        self._seeds = [rand_seed()]
        
    @property
    def seeds(self):
        return self._seeds
        
    def mutate(self):
        self._seeds.append(rand_seed())
        
    def crossover(self, other):
        # TODO: experiment with one-point/two-point crossover later
        raise NotImplementedError

In [11]:
g = Genotype()
print("Initially...")
print("g.seeds =", g.seeds)

for _ in range(10):
    g.mutate()

print("After mutations...")
print("g.seeds =", g.seeds)

Initially...
g.seeds = [100697675]
After mutations...
g.seeds = [100697675, 2068154422, 1873692604, 1110660976, 1354455261, 1702619479, 783287689, 1264158678, 510246554, 295117216, 1905590102]


In [8]:
def decode(genome, tmpl_model, sigma):
    model = deepcopy(tmpl_model)
    for seed in genome.seeds:
        # NOTE: in the paper, the first seed is used for initialization,
        # but in such case, every individual in the first generation is
        # initialized around zero; we probably don't want that.
        #
        # Instead, we're going to assume that all individuals are already
        # initialized with a better initialization method, e.g., Xavier.
        torch.manual_seed(seed)
        for param in model.parameters():
            param.data.add_(torch.randn_like(param) * sigma)
    return model

Ooh, looks like we're going to have to implement the phenotype before testing `deocde`.

## Phenotype

In [13]:
class NatureDQN(nn.Module):
    def __init__(self, in_channels=4, act_dim=18):
        super(NatureDQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        torch.nn.init.xavier_uniform(self.conv1.weight)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        torch.nn.init.xavier_uniform(self.conv3.weight)
        self.fc4 = nn.Linear(7 * 7 * 64, 512)
        torch.nn.init.xavier_uniform(self.fc4.weight)
        self.fc5 = nn.Linear(512, act_dim)
        torch.nn.init.xavier_uniform(self.fc5.weight)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc4(x))
        return self.fc5(x)

In [15]:
summary(NatureDQN().to(DEVICE), (4, 84, 84))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           8,224
            Conv2d-2             [-1, 64, 9, 9]          32,832
            Conv2d-3             [-1, 64, 7, 7]          36,928
            Linear-4                  [-1, 512]       1,606,144
            Linear-5                   [-1, 18]           9,234
Total params: 1,693,362
Trainable params: 1,693,362
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.11
Forward/backward pass size (MB): 0.17
Params size (MB): 6.46
Estimated Total Size (MB): 6.73
----------------------------------------------------------------
